# Policy Iteration
* Dynamic programming 중에 하나
* DP 는 MDP에 관한 모든 정보를 알고 있다고 가정
* Prediction 문제 (value function 찾는 문제)
  * 입력: MDP $<S, A, P, R, \gamma>$ 와 policy $\pi$
  * 또는 MRP $<S, P^{\pi}, R^{\pi}, \gamma>$
  * 출력: value function $v_{\pi}$
* Control 문제 (policy 찾는 문제)
  * 입력: MDP $<S, A, P, R, \gamma>$
  * 출력: optimal value function $v_*$
  * 그리고 optimal policy $\pi_*$

policy evaluation + policy improvement

## Frozen Lake
* Frozen lake 환경을 세팅
* Gym 에서 discrete 한 environment 쓸만한 놈이 이거라서 가져옴
* 'S' 타일에서 시작
* Action으로 'Up', 'Left', 'Right', 'Down'을 줄 수 있음
* 에피소드는 'H' 타일에 빠져서 reward 0 (죽음) 얻거나, 'G' (goal) 타일로 가서 reward 1 (점수) 얻거나

In [1]:
import gym
import numpy as np

```is_slippery``` 를 True, False에 따라
* True: action에 따라 확정적으로 이동하지 않고, 확률적으로 action 말고 다른데로 가버림 (frozen lake라 미끄러워서 그렇다고 함...)
* False: action 에 따라 확정적으로 이동

In [2]:
env = gym.make('FrozenLake-v0', is_slippery=False)

In [3]:
S = env.observation_space.n
A = env.action_space.n
print('state space:', np.arange(S))
print('action space:', np.arange(A))

state space: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
action space: [0 1 2 3]


In [4]:
print("랜덤하게 움직여서 겜 한판 돌려보자!")
s = env.reset()
env.render()


while True:
    s, r, done, _ = env.step(env.action_space.sample())
    env.render()
    if done:
        break

랜덤하게 움직여서 겜 한판 돌려보자!

SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG


## constant
* $\gamma$ : discount factor
* $\theta$ : policy evaluation threshold

In [5]:
GAMMA = .9 # discount factor
EPISODES = 1000 # policy evaluation 끝나고 transition matrix 업데이트 할 횟수
THETA = 1e-4 # policy evaluation threshold

## variables

* $\pi$ : policy. state에 따라 어떤 action 취할지 정함

In [6]:
# init policy, S x A x 1
pi = np.ones((S, A))
pi /= A
pi

array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]])

* 리워드

In [7]:
# init rewrads, S x A x 1
rewards = np.zeros((S, A))

* transition (model 이라고도 함)

In [8]:
# init transition matrix, A x S x S' x 1
Pss = np.zeros((A, S, S))

* value function : 이 예제에서는 state-value function 을 다룸

In [9]:
# init value function, S x 1
valuefunction = np.ones((S)) / S

In [10]:
# return list
return_list = []

## policy evaluation
* prediciton 문제
* 어떤 policy 가 주어졌다 치고, value function 찾기
* Solution: Bellman 기대식을 iterative 하게 적용
* synchronous 로 업뎃
  * 각 iteration $k+1$
  * 모든 states $s\in S$에 대해서
  * $v_k(s')$ 로 부터 $v_{k+1}(s)$를 업뎃
  * 이 때 $s'$는 $s$로 부터 뒤따라 나온 state

(asynchronous 방법은 나중에...)

($v_{\pi}$ 의 convergence 증명은 Sutton 책에...)

* Bellman expectation equation

\begin{align}
v_{k+1}(s) & = \sum_{a\in A} \pi (a|s) \big( R^a_s + \gamma \sum_{s'\in S} P^a_{ss'} v_k (s') \big)\\
v^{k+1} & = R^{\pi} + \gamma P^{\pi} v^k
\end{align}

* 그냥 별 쓰잘데기 없는 이상한 policy를 써서라도 v를 업뎃함
* k가 k+1 로 진행됨에 따라 reward는 정확하기 때문에 v가 올바른 값이 얻어지게 됨

### policy evaluation algorithm

Input $\pi$, the policy to be evaluated</br>
Algorithm parameter: a small threshold $\theta > 0$ determining accuracy of estimation</br>
Initialize $V(s)$, for all $s\in S^+$, arbitrarily except that $V (terminal) = 0$

Loop:</br>
&nbsp;&nbsp;&nbsp;$\Delta\leftarrow 0$</br>
&nbsp;&nbsp;&nbsp;Loop for each $s\in S$:</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$v\leftarrow V(s)$</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$V(s)\leftarrow \sum_a \pi (a|s) \sum_{s',r}p(s', r|s, a)[r+\gamma V(s')]$</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\Delta \leftarrow \max (\Delta, |v-V(s)|)$</br>
until $\Delta < \theta$

* 아래 구현에서는 $r$에 $p(s',r|s,a)$ 곱하지 않고 그냥 $r$ 씀
* 즉 아래 식으로 업뎃
$$
v^{k+1} = R^\pi + \gamma P^\pi v^k
$$

In [11]:
action = np.arange(A)

def get_action(state):
    """
    args:
        state: 상태
    return: 
        policy 의 확률에 따라 취할 수 있는 action 중에서 하나 구해서 리턴
    """
    return np.random.choice(action, size=None, p=pi[state])

def get_transition(episodes):
    """
    args:
        episodes 수
    return: 
        None
        에피소드 수 주어진 만큼 environment 돌아다니면서 transition matrix 채움
    """
    global rewards, return_list
    
    _rewards = np.zeros(rewards.shape)
    
    for _ in range(episodes):
        state = env.reset()
        while True:
            action = get_action(state)
            state_, _reward, done, _ = env.step(action)
            
            Pss[action][state][state_] += 1 # cumulate again and again
            _rewards[state][action] += _reward
            
            if done:
                break
            
            state = state_
                
    # average rewards
    _rewards /= episodes
    rewards = _rewards
    return_list.append(np.sum(rewards))

In [12]:
# POLICY EVALUATION

# 에피소드 수 만큼 돌면서 transition matrix를 채움
get_transition(episodes=100)

while True:
    # loop 멈출 현재의 threshold
    delta = .0

    PssV = np.zeros((A, S))
    pss_sum = np.sum(Pss)
    for a in range(A):
        for s in range(S):
            _sum = 0.
            for s_ in range(S):
                _sum += Pss[a][s][s_]/pss_sum * valuefunction[s_]
            PssV[a][s] = _sum
            
    PssV *= GAMMA
    
    Rsum = np.zeros((S, A))
    for s in range(S):
        for a in range(A):
            Rsum[s][a] += rewards[s][a] + PssV[a][s]
    
    newV = np.zeros((S))
    for s in range(S):
        for a in range(A):
            newV[s] += pi[s][a] * Rsum[s][a]
            
    delta = max(delta, np.sum(np.abs(valuefunction - newV)))
    
    if delta < THETA:
        break
        
    valuefunction = np.array(newV)
    
    # 그 담번엔 EPISODES번씩
    get_transition(EPISODES)

In [13]:
valuefunction

array([7.47957265e-15, 9.22095298e-14, 7.95191468e-12, 2.11385951e-14,
       1.18045103e-13, 0.00000000e+00, 2.45071906e-09, 0.00000000e+00,
       1.05244224e-11, 4.08203959e-09, 2.69967727e-06, 0.00000000e+00,
       0.00000000e+00, 2.12303247e-06, 2.75156677e-03, 0.00000000e+00])

In [14]:
rewards

array([[0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.011, 0.   ],
       [0.   , 0.   , 0.   , 0.   ]])

## policy improvement
* control 문제
* value function 이 optimal 이라 치고 policy 찾기 문제

* Policy $\pi$가 주어졌을 때
  * policy $\pi$ 를 **evaluate**
  $$
  v_{\pi}(s) = \mathbb{E} \big[ R_{t+1} + \gamma R_{t+2} + \cdots | S_t = s \big]
  $$
  * $v_{\pi}$ 에 따라서 policy 를 greedy 하게 **improve**
  $$
  \pi' = \mbox{greedy}(v_{\pi})
  $$
  
* 작은 그리드 위에서 goal 찾기 (예: frozen lake에서 action 이 random 아니면) 는 걍 improved policy 한 번 하면 바로 optimal policy $\pi^*$가 찾아짐
* 근데 일반적으로는 improvement / evaluation 을 몇 iteration 돌려주어야 함
* **Policy iteration** 이 항상 $\pi^*$에 converge 함
* 즉 되도 않은 value function 이랑 되도 않은 policy 로 시작해도 policy iteration 하면 optimal policy 찾아진다는 것

### Policy improvement 증명
(improvement 로 진짜 더 나은 policy 찾을 수 있음?)
* deterministic policy $a=\pi (s)$ 에 대해서
* greedy 하게 policy 를 *improve* 가능
$$
\pi' (s) = \arg \max_{a\in A} q_{\pi}(s, a)
$$
* 이를 통해 한 스텝으로 어떤 state $s$ 로부터 value 를 improve 가능
$$
q_{\pi} (s, \pi'(s)) = \max_{a\in A} q_{\pi} (s, a) \ge q_{\pi} (s, \pi (s)) = v_{\pi} (s)
$$
* 그래서 value function 을 improve 함, $v_{\pi'} (s) \ge v_{\pi} (s)$
\begin{align}
v_{\pi} (s) & \le q_{\pi}(s, \pi'(s)) = \mathbb{E}_{\pi'} [R_{t+1} + \gamma v_{\pi} (S_{t+1}) | S_t = s]\\
& \le \mathbb{E}_{\pi'} [R_{t+1} + \gamma q_{\pi}(S_{t+1},\pi'(S_{t+1}))|S_t = s]\\
& \le \mathbb{E}_{\pi'} [R_{t+1} + \gamma R_{t+2} + \gamma^2 q_{\pi}(S_{t+2}, \pi'(S_{t+2}))|S_t = s]\\
& \le \mathbb{E}_{\pi'} [R_[t+1] + \gamma R_{t+2} + \cdots | S_t = s] = v_{\pi'}(s)
\end{align}
* 만약에 improvement 가 멈추면
$$
q_{\pi} (s, \pi'(s)) = \max_{a\in A} q_{\pi} (s, a) = q_{\pi} (s, \pi (s)) = v_{\pi} (s)
$$
* 그러면 Bellman 최적등식이 만족
$$
v_{\pi} (s) = \max_{a\in A} q_{\pi} (s, a)
$$
* 그래서 모든 $s\in S$ 에 대해 $v_{\pi}(s) = v_* (s)$
* 그래서 $\pi$는 optimal policy

### policy improvement algorithm

$policy-stable \leftarrow true$</br>
For each $s\in S$:</br>
&nbsp;&nbsp;&nbsp;&nbsp;$old-action \leftarrow \pi(s)$</br>
&nbsp;&nbsp;&nbsp;&nbsp;$\pi(s) \leftarrow \arg\max_a \sum_{s',r} p(s', r | s, a)[r + \gamma V(s')]$</br>
&nbsp;&nbsp;&nbsp;&nbsp;If $old-action \neq \pi(s)$, then $policy-stable\leftarrow false$</br>
If $policy-stable$, then stop and return $V\approx v_*$ and $]pi \approx \pi_*$

In [15]:
# POLICY IMPROVEMENT

# old-action 과 new-action 비교용 임시저장공간. 이 셀에서는 사용X
old_action = np.array(pi)

PssV = np.zeros((A, S))

# Transition matrix 에 count 저장된거 이걸로 나눠주기 해서 확률로 바꾸는 작업
pss_sum = np.sum(Pss) # 실은 axis=-1로 해주어야 함
for a in range(A):
    for s in range(S):
        _sum = 0.
        for s_ in range(S):
            _sum += Pss[a][s][s_]/pss_sum * valuefunction[s_]
        PssV[a][s] = _sum

# 위 수식에서 감마 곱하기 까지 작업
PssV *= GAMMA

# 리워드도 더해주는 작업, 저 위에 알고리즘과 약간 다른 수식 사용함
for s in range(S):
    for a in range(A):
        Rsum[s][a] += rewards[s][a] + PssV[a][s]
        
# state별로 max value (greedy) 인 action 취하도록 policy 수정
new_action = np.zeros((S, A))
for s in range(S):
    if np.min(Rsum[s]) != np.max(Rsum[s]):
        new_action[s][np.argmax(Rsum[s])] = 1.
    else:
        new_action[s] = pi[s]

In [16]:
pi

array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]])

## policy iteration

In [17]:
# 첨엔 100번
get_transition(100)

while True:

    '''
    POLICY EVALUATION
    '''

    while True:
        delta = .0

        PssV = np.zeros((A, S))
        pss_sum = np.sum(Pss)
        for a in range(A):
            for s in range(S):
                _sum = 0.
                for s_ in range(S):
                    _sum += Pss[a][s][s_]/pss_sum * valuefunction[s_]
                PssV[a][s] = _sum

        PssV *= GAMMA

        Rsum = np.zeros((S, A))
        for s in range(S):
            for a in range(A):
                Rsum[s][a] += rewards[s][a] + PssV[a][s]

        newV = np.zeros((S))
        for s in range(S):
            for a in range(A):
                newV[s] += pi[s][a] * Rsum[s][a]

        delta = max(delta, np.sum(np.abs(valuefunction - newV)))

        if delta < THETA:
            break

        valuefunction = np.array(newV)

        get_transition(EPISODES)
        
        return_list.append(np.sum(rewards))



    """
    POLICY IMPROVEMENT
    """

    policy_stable = True

    old_action = np.array(pi)

    PssV = np.zeros((A, S))
    pss_sum = np.sum(Pss)
    for a in range(A):
        for s in range(S):
            _sum = 0.
            for s_ in range(S):
                _sum += Pss[a][s][s_]/pss_sum * valuefunction[s_]
            PssV[a][s] = _sum

    PssV *= GAMMA

    new_action = np.zeros((S, A))
    for s in range(S):
        for a in range(A):
            Rsum[s][a] += rewards[s][a] + PssV[a][s]
    for s in range(S):
        if np.min(Rsum[s]) != np.max(Rsum[s]):
            new_action[s][np.argmax(Rsum[s])] = 1.
        else:
            new_action[s] = pi[s]


    if not (old_action == new_action).all():
        policy_stable = False
        pi = np.array(new_action)
        
    if policy_stable:
        print("We have pi* and v*!")
        print("pi*")
        print(pi)
        print("v*")
        print(valuefunction)
        break

We have pi* and v*!
pi*
[[0.   1.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [1.   0.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.   1.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.   0.   1.   0.  ]
 [0.   0.   1.   0.  ]
 [0.   1.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.   0.   1.   0.  ]
 [0.   0.   1.   0.  ]
 [0.25 0.25 0.25 0.25]]
v*
[3.33897947e-05 2.57076660e-09 3.62931964e-07 5.97495331e-10
 2.39477736e-04 0.00000000e+00 1.12459477e-04 0.00000000e+00
 1.86470601e-03 1.50064188e-02 1.22379743e-01 0.00000000e+00
 0.00000000e+00 3.88706446e-04 1.00000000e+00 0.00000000e+00]


In [18]:
print("Policy iteration으로 찾은 policy에 따라 겜 해보자!")
s = env.reset()
env.render()

while True:
    s, r, done, _ = env.step(get_action(s))
    env.render()
    if done:
        break
        
print('reward:',r)

Policy iteration으로 찾은 policy에 따라 겜 해보자!

SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
reward: 1.0


In [19]:
# for a in range(A):
#     for s in range(S):
#         for s_ in range(S):
#             print(f"(a:{a}, s:{s}) -> s':{s_} : {Pss[a][s][s_]}")

In [20]:
s = env.reset()
cnt_success = 0
trial = 1000

for _ in range(trial):
    while True:
        s, r, done, _ = env.step(get_action(s))
        cnt_success += r
        if done:
            s = env.reset()
            break
        
print(f"policy iteration 으로 풀었을 때, 에이전트 퍼포먼스: {trial}회 중 {cnt_success:n}회 성공 -> {cnt_success/trial * 100:.2f}%")

policy iteration 으로 풀었을 때, 에이전트 퍼포먼스: 1000회 중 1000회 성공 -> 100.00%


In [21]:
s = env.reset()
cnt_success = 0
trial = 1000

for _ in range(trial):
    while True:
        s, r, done, _ = env.step(env.action_space.sample())

        cnt_success += r

        if done:
            s = env.reset()
            break
        
print(f"아무곳이나 가는 에이전트 퍼포먼스: {trial}회 중 {cnt_success:n}회 성공 -> {cnt_success/trial * 100:.2f}%")

아무곳이나 가는 에이전트 퍼포먼스: 1000회 중 8회 성공 -> 0.80%
